In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
mpl.rcParams["figure.dpi"] = 300
from skimage.io import imread

plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100  # 200 e.g. is really fine, but slower
plt.style.use("classic")
plt.style.use("seaborn-white")

plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100  # 200 e.g. is really fine, but slower

mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["ps.fonttype"] = 42
plt.rcParams["font.family"] = "Arial"

import re

import pandas as pd
import scanpy as sc

sc.set_figure_params(dpi=200)


import seaborn as sns
import skimage
from morphometrics.explore.cluster import cluster_features
from morphometrics.explore.dimensionality_reduction import pca
from morphometrics.utils.anndata_utils import table_to_anndata
from skimage.measure import label
from tqdm import tqdm

rng = np.random.default_rng(42)
import scipy.spatial.distance as distance


def colorFader(
    c1, c2, mix=0
):  # fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1 - mix) * c1 + mix * c2)


cm = 1 / 2.54  # centimeters in inches
import matplotlib

In [ ]:
# Load data + Pseudotime
measurement_data = sc.read_h5ad(
    "anndatas/actin_morphometrics_extended_figure_7_fine_grain.h5ad"
)

In [ ]:
sc.pl.umap(
    measurement_data,
    color="leiden",
    size=14,
    title="",
    frameon=False,
    legend_fontsize="x-small",
)

In [ ]:
# Color palette
import met_brewer

colors = met_brewer.met_brew(
    name="Johnson",
    n=len(np.unique(measurement_data.obs["leiden"])),
    brew_type="continuous",
)
cmap_brewer_umap = matplotlib.colors.ListedColormap(colors, name="from_list", N=None)

In [ ]:
leiden_age = pd.DataFrame(measurement_data.obs.groupby("leiden")["Day"].mean())
leiden_age["cluster"] = leiden_age.index
leiden_age_sorted = leiden_age.sort_values("Day")
# Color palette
color_pink_midnight_blue = matplotlib.colors.LinearSegmentedColormap.from_list(
    "", ["#b3107a", "#f7f7f7", "#191970"]
)

In [ ]:
marker_label = "actin"

In [ ]:
sc.pl.umap(
    measurement_data,
    color="leiden",
    legend_fontsize=7,
    palette=colors,
    size=14,
    title="",
    frameon=False,
    # save=f"_{marker_label}_clusters_fine_grain.pdf",
)

sc.pl.paga(
    measurement_data,
    title="",
    labels=len(np.unique(measurement_data.obs["leiden"])) * [""],
    node_size_scale=1,
    threshold=0.1,
    frameon=False,
    fontsize=5,
    cmap=cmap_brewer_umap,
    edge_width_scale=0.2,
    # save=f"_{marker_label}_clusters_fine_grain.pdf",
)


sc.pl.paga(
    measurement_data,
    color=["volume_raw"],
    title="",
    labels=len(np.unique(measurement_data.obs["leiden"])) * [""],
    node_size_scale=1,
    threshold=0.1,
    fontsize=6,
    frameon=False,
    edge_width_scale=0.2,
    # save=f"_{marker_label}_volume_fine_grain.pdf",
)

sc.pl.paga(
    measurement_data,
    color=["Axis_length_ratio_raw"],
    title="",
    labels=len(np.unique(measurement_data.obs["leiden"])) * [""],
    node_size_scale=1,
    threshold=0.1,
    fontsize=6,
    frameon=False,
    edge_width_scale=0.2,
    # save=f"_{marker_label}_axis_ratio_fine_grain.pdf",
)

sc.pl.paga(
    measurement_data,
    color=["Day"],
    title="",
    labels=len(np.unique(measurement_data.obs["leiden"])) * [""],
    node_size_scale=1,
    threshold=0.1,
    fontsize=6,
    frameon=False,
    edge_width_scale=0.2,
    # save=f"_{marker_label}_fine_grain_average_age.pdf",
)

In [ ]:
leiden_age_sorted["cluster"][0]

In [ ]:
sc.pl.umap(
    measurement_data,
    color="dpt_pseudotime",
    legend_fontsize=7,
    palette=colors,
    size=14,
    title="",
    frameon=False,
    # save=f"_{marker_label}_clusters_fine_grain.pdf",
)

In [ ]:
sc.pl.paga_path(
    measurement_data,
    n_avg=20,
    nodes=list(leiden_age_sorted["cluster"].astype(str)),
    keys=measurement_data.var_names,
    color_maps_annotations={"Day": "Greys"},
    annotations=("dpt_pseudotime", "Day"),
    normalize_to_zero_one=True,
    color_map=color_pink_midnight_blue,
    # save="_paga_heatmap_fine_grain.pdf",
    show_node_names=False,
)